# Facebook Ad Statistical Analysis
This notebook will contain three different data tables. The fb_time table has impressions by hour of day as well as the cost per mile (CPM). The fb_demo table has the demographics of viewers. It contains age, gender, reach, impressions, and CPM. The last table is fb_clicks, which includes the click amount, reach, impressions, conversions (in this case, a conversion was someone filling out a form on the website, giving their email and other PID for more information on the ad subject), and costs again.

In [1]:
from scipy import stats
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io

In [2]:
upload = files.upload()

Saving Facebook_ads_campaign_v2.csv to Facebook_ads_campaign_v2.csv
Saving facebook_demographics_v2.csv to facebook_demographics_v2.csv
Saving facebook_time_v2.csv to facebook_time_v2.csv


In [3]:
fb_demo = pd.read_csv(io.BytesIO(upload['facebook_demographics_v2.csv'])) ##importing the files
fb_time = pd.read_csv(io.BytesIO(upload['facebook_time_v2.csv']))
fb_clicks = pd.read_csv(io.BytesIO(upload['Facebook_ads_campaign_v2.csv']))

# Cleaning and Formatting Data

In [5]:
# You may notice the tables all have v2. This is because I already cleaned them up a bit and got rid of some unncessecary information. Lets take a look at the tables
fb_demo.head()

,Age,Gender,Reach,Impressions,Frequency,Attribution setting,Result type,Amount spent (USD),Reporting starts,Reporting ends
0,45-54,male,678,1202,1.772861,7-day click or 1-day view,NaN,21.22,2023-08-30,2023-09-29
1,35-44,male,707,1248,1.765205,7-day click or 1-day view,NaN,20.03,2023-08-30,2023-09-29
2,55-64,male,605,1149,1.899174,7-day click or 1-day view,NaN,19.59,2023-08-30,2023-09-29
3,55-64,female,246,449,1.825203,7-day click or 1-day view,NaN,14.47,2023-08-30,2023-09-29
4,45-54,female,191,336,1.759162,7-day click or 1-day view,NaN,9.58,2023-08-30,2023-09-29


In [6]:
fb_time.head()

,Time of day (ad account time zone),Impressions,Attribution setting,Result type,Amount spent (USD),Cost per mile,Reporting starts,Reporting ends
0,07:00:00 - 07:59:59,398,7-day click or 1-day view,NaN,8.64,21.708543,2023-08-30,2023-09-29
1,10:00:00 - 10:59:59,403,7-day click or 1-day view,NaN,8.11,20.124069,2023-08-30,2023-09-29
2,15:00:00 - 15:59:59,362,7-day click or 1-day view,NaN,7.94,21.933702,2023-08-30,2023-09-29
3,12:00:00 - 12:59:59,294,7-day click or 1-day view,NaN,7.61,25.884354,2023-08-30,2023-09-29
4,08:00:00 - 08:59:59,362,7-day click or 1-day view,NaN,7.08,19.558011,2023-08-30,2023-09-29


In [8]:
fb_clicks.head()

,Campaign name,Attribution setting,Results,Link clicks,Bonce count,Bounce rate,Reach,Impressions,Cost per results,Amount spent (USD)
0,Link Click,7-day click or 1-day view,74,239,165,0.690377,2990,5198,0.12018,28.7
1,Link Click,7-day click or 1-day view,74,239,165,0.690377,2990,5198,19.31000,100.4


I did not mention previously that the fb_clicks table includes bounce rate. That was a simple calculation I did in excel to find the rate at which people leave the the website without converting. This is complement to the conversion rate, which appears to be approximately 30%. I will go deeper into the rates later. There is also a cost per results column. For this ad the marketer did CPM (Cost per mile) but I used their typical CPC (Cost per click) to calculate what the price would have been using CPC instead of CPM. Of course the CPC could have been more than what I used, but it typically ranges from 30 cents to a few dollars. As for the financial analysis, I will be compeleting that in a seperate notebook, along with other ad campaigns.

Before I get started on the analysis, I do want to clean some stuff I did not do in excel just for clarity.

In [9]:
fb_time.rename(columns={'Time of day (ad account time zone)': 'Time (24hr)'}, inplace=True)

In [10]:
fb_time.sort_values(by='Time (24hr)', inplace=True)


In [12]:
fb_time.drop('Result type', axis=1, inplace=True)

In [13]:
fb_time.head()

,Time (24hr),Impressions,Attribution setting,Amount spent (USD),Cost per mile,Reporting starts,Reporting ends
18,00:00:00 - 00:59:59,63,7-day click or 1-day view,1.59,25.238095,2023-08-30,2023-09-29
20,01:00:00 - 01:59:59,23,7-day click or 1-day view,0.78,33.913043,2023-08-30,2023-09-29
22,02:00:00 - 02:59:59,18,7-day click or 1-day view,0.49,27.222222,2023-08-30,2023-09-29
23,03:00:00 - 03:59:59,29,7-day click or 1-day view,0.48,16.551724,2023-08-30,2023-09-29
19,04:00:00 - 04:59:59,78,7-day click or 1-day view,1.51,19.358974,2023-08-30,2023-09-29


In [14]:
fb_demo.drop('Result type', axis=1, inplace=True)

In [15]:
fb_demo.head()

,Age,Gender,Reach,Impressions,Frequency,Attribution setting,Amount spent (USD),Reporting starts,Reporting ends
0,45-54,male,678,1202,1.772861,7-day click or 1-day view,21.22,2023-08-30,2023-09-29
1,35-44,male,707,1248,1.765205,7-day click or 1-day view,20.03,2023-08-30,2023-09-29
2,55-64,male,605,1149,1.899174,7-day click or 1-day view,19.59,2023-08-30,2023-09-29
3,55-64,female,246,449,1.825203,7-day click or 1-day view,14.47,2023-08-30,2023-09-29
4,45-54,female,191,336,1.759162,7-day click or 1-day view,9.58,2023-08-30,2023-09-29


# Statistical Analysis

for the analysis, I will be calculating the statistical significance of the metrics asked for by the marketer. These metrics were included on a [dashboard](https://public.tableau.com/app/profile/arianna.langton5684/viz/FacebookAdCampaignReachAnalysis/Dashboard1) I created on Tableau. They are the following:

*   Male vs Female impressions vs reach rate
*   Male vs Female total impressions and reach
*   Age that recieved most impressions
*   Time of day that recieved most impressions
*   Confidence interval for the conversion rate (CVR)
*   Confidence interval for impressions to conversions (Imp CR)
*   Confidence interval for click through rate (CTR)

I will go down the list in order to do the analysis.





Since I am not comparing averages, I will do a 2-proportion z-test to compare the success rate (reach) of male vs female viewers (impressions).

 ### Null Hypothesis : there is no difference in reach percent in male vs female
 ### Alternative Hypothesis : there is a difference in reach percent in male v female

 Now I will quickly aggregate the data to get the sums of male and female reach and impressions.

In [26]:
fb_demo_gender = fb_demo.groupby('Gender').agg({'Reach': 'sum', 'Impressions': 'sum'})

In [27]:
fb_demo_gender.head()

,Reach,Impressions
Gender,,
female,675,1131
male,2288,4027
unknown,27,40


In [25]:

# success counts and total viewers
male_successes = 2288  # Number of successful male viewers
female_successes = 675  # Number of successful female viewers
male_total = 4027  # Total number of male viewers
female_total = 1131  # Total number of female viewers

# Counts of successes for male and female
successes = np.array([female_successes, male_successes])

# Total number of trials for male and female
n = np.array([female_total, male_total])

# Perform the z-test for proportions
z_stat, p_value = proportions_ztest(successes, n)

# Print results
print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")

# Interpretation of results
alpha = 0.05  # Industry standard significance level of 5%

if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between male and female success rates.")
elif p_value < 0.1:
    print("Moderate evidence against the null hypothesis: There is a moderate difference between male and female success rates.")
else:
    print("Fail to reject the null hypothesis: No significant difference between male and female success rates.")


Z-statistic: 1.7220112776987127
P-value: 0.08506747556619909
Moderate evidence against the null hypothesis: There is a moderate difference between male and female success rates.


Typically I would stick with the industry standard of only using p < 0.05 to state significance. However, given the small size of the data set and the low cost, it is safe to say there is still some statistical difference between male and female succes rates and use it for further planning. However, I would suggest gathering more data before making any extreme choices.

Now I will do the hypothesis test for male vs female impressions and reach count.

### Null Hypothesis 1: there is no difference in reach in male vs female
### Alternative Hypothesis 1: there is a difference in reach in male vs female


### Null Hypothesis 2: there is no difference in impressions in male vs female
### Alternative Hypothesis 2: there is a difference in impressions in male v female

In [24]:
#z test for male vs female reach and impressions
# Example total numbers
male_reach = 2288  # Total number of male reach
female_reach = 675  # Total number of female reach
total_reach = male_reach + female_reach  # Combined total viewers

male_impressions = 4027  # Total number of male impressions
female_impressions = 1131  # Total number of female impressions
total_impressions = male_impressions + female_impressions  # Combined total impressions

# Count of male and female viewers (this is like the "success" counts)
reach_counts = np.array([male_reach, female_reach])
imp_counts = np.array([male_impressions, female_impressions])

# Number of groups (two groups: male and female)
reach_n = np.array([total_reach, total_reach])  # Number of total reach in each group
impressions_n = np.array([total_impressions, total_impressions])  # Number of total impressions in each group

# Perform a z-test for proportions
stat, p_value = proportions_ztest(reach_counts, reach_n)
stat_impressions, p_value_impressions = proportions_ztest(imp_counts, impressions_n)

# Print results
print(f"Z-statistic: {stat}")
print(f"P-value: {p_value}")

print(f"Z-statistic: {stat_impressions}")
print(f"P-value: {p_value_impressions}")

# Interpret the result
alpha = 0.05  # Significance level of 5%

if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between male and female reach.")
else:
    print("Fail to reject the null hypothesis: No significant difference between male and female reach.")

if p_value_impressions < alpha:
    print("Reject the null hypothesis: There is a significant difference between male and female impressions.")
else:
    print("Fail to reject the null hypothesis: No significant difference between male and female impressions.")


Z-statistic: 41.90670737302639
P-value: 0.0
Z-statistic: 57.02599694225984
P-value: 0.0
Reject the null hypothesis: There is a significant difference between male and female reach.
Reject the null hypothesis: There is a significant difference between male and female impressions.


While it is uncertain is the reach to impression rate is different for males and females, it is clear from the hypothesis test that the amount of viewers by gender is statistially different, as the p value is 0. It is safe to say that men were the primary viewers of this ad in both impressions and reach.

Now, I will analyze age.

### Null Hypothesis: there is no difference in age for impressions.
### Alternative Hypothesis: At least one group has a difference in impressions.

In [28]:
fb_demo_age = fb_demo.groupby('Age').agg({'Reach': 'sum', 'Impressions': 'sum'})

In [58]:
fb_demo_age.head(6)

,Reach,Impressions
Age,,
18-24,17,24
25-34,178,262
35-44,891,1529
45-54,878,1551
55-64,856,1606
65+,170,226


Now that we have the values it is clear there is a significant difference in age groups. To compare these I would want to run a one way ANOVA test, but since these are not averages and I do not have any variance values I will have to stick with simple comparisons instead. Lets rename the colums for simplicity.

In [57]:
fb_demo_age.rename(index={'18-24': 'one', '25-34': 'two', '35-44':'three','45-54':'four','55-64':'five','65+':'six'})

,Reach,Impressions
Age,,
one,17,24
two,178,262
three,891,1529
four,878,1551
five,856,1606
six,170,226


In [67]:
five_to_six = round((1606 - 226) / 1606, 2)* 100
five_to_four = round((1606 - 1551) / 1606, 2)* 100
five_to_three = round((1606 - 1529) / 1606,2)* 100
five_to_two = round((1606 - 262) / 1606, 2) * 100
five_to_one = round((1606 - 24) / 1606, 2) * 100

In [68]:
print(f'The percent difference between the column with the most views and the rest ranges {five_to_one}% and {five_to_three}%')

The percent difference between the column with the most views and the rest ranges 99.0% and 5.0%


Clearly, if there if there is a percent difference of 99% it is safe to assume they would be statistically different. The data shows that the age group with the most views is between 55-64, so they would be a good group to target going forward.

Now, I will quickly analyze time by aggregating it and sorting it desceding to see what hours get the most views.

In [71]:
fb_time.groupby('Time (24hr)').agg({'Impressions': 'sum'}).sort_values(by='Impressions', ascending=False).head()

,Impressions
Time (24hr),
10:00:00 - 10:59:59,403
07:00:00 - 07:59:59,398
15:00:00 - 15:59:59,362
08:00:00 - 08:59:59,362
11:00:00 - 11:59:59,340


From this we see that 10-11am, 7-8am, 3-4pm, 8-9am, and 11-12pm get the most views. From this, I would guess that it is getting viewed before work at school (7-8am, 8-9am), during lunch breaks (10-11am, 11-12pm) and then after work or school (3-4pm). This seems to go along with our age range, as elderly people tend to be up early and go to bed early, which is why we do not see any late times in our top five.

However, there is not a huge difference in impressions among the top five, so it cannot currently be decided which of the five is the most successful time. I would suggest to gather more data before making a decision.

Finally, I will be calcuating confidence intervals for conversion rate (CVR), impression to conversion rate (Imp CVR) and click through rate (CTR). These will all be done using a 95% confidence interval.

In [83]:
from scipy.stats import norm

def calculate_conversion_confidence_interval(conversions, total_views, confidence_level=0.95):
    """
    Calculates the confidence interval for a conversion rate.

    Args:
        conversions (int): Number of conversions.
        total_visitors (int): Total number of visitors.
        confidence_level (float, optional): Confidence level (e.g., 0.95 for 95%). Defaults to 0.95.

    Returns:
        tuple: A tuple containing the lower and upper bounds of the confidence interval.
    """
    if total_views <= 0:
        raise ValueError("Total views must be greater than zero.")

    p = conversions / total_views  # Conversion rate

    if p == 0 or p == 1:
      return (0,1)

    z = norm.ppf(1 - (1 - confidence_level) / 2)  # Z-score for the given confidence level

    standard_error = np.sqrt((p * (1 - p)) / total_views)
    margin_of_error = z * standard_error

    lower_bound = p - margin_of_error
    upper_bound = p + margin_of_error

    return f"({round(lower_bound * 100,2)}%, {round(upper_bound * 100, 2)}%)"

In [74]:
fb_clicks.head()

,Campaign name,Attribution setting,Results,Link clicks,Bonce count,Bounce rate,Reach,Impressions,Cost per results,Amount spent (USD)
0,Link Click,7-day click or 1-day view,74,239,165,0.690377,2990,5198,0.12018,28.7
1,Link Click,7-day click or 1-day view,74,239,165,0.690377,2990,5198,19.31000,100.4


I will quickly rename the results column to conversions to clear up any confusion.

In [75]:
fb_clicks.rename(columns={'Results': 'Conversions'}, inplace=True)

In [76]:
fb_clicks

,Campaign name,Attribution setting,Conversions,Link clicks,Bonce count,Bounce rate,Reach,Impressions,Cost per results,Amount spent (USD)
0,Link Click,7-day click or 1-day view,74,239,165,0.690377,2990,5198,0.12018,28.7
1,Link Click,7-day click or 1-day view,74,239,165,0.690377,2990,5198,19.31000,100.4


In [85]:
conversions = 74
link_clicks = 239
confidence_interval = calculate_conversion_confidence_interval(conversions, link_clicks)
print(f"The conversion rate confidence interval is: {confidence_interval}")

The conversion rate confidence interval is: (25.1%, 36.82%)


In [99]:
lower_lim_conv = 239 * 0.251
upper_lim_conv = 239 * 0.3682
print(f"The 95% confidence interval for amount of conversions is then between {round(lower_lim_conv,0)} and {round(upper_lim_conv,0)} people")

The 95% confidence interval for amount of conversions is then between 60.0 and 88.0 people


In [88]:
impressions = 5198
confidence_interval = calculate_conversion_confidence_interval(conversions, impressions)
print(f"The impression to conversion rate confidence interval is: {confidence_interval}")

The impression to conversion rate confidence interval is: (1.1%, 1.75%)


The average impression to conversion rate is between 2-5%, so this ad failed to reach the average rate.

In [90]:
confidence_interval = calculate_conversion_confidence_interval(link_clicks, impressions)
print(f"The click through rate confidence interval is: {confidence_interval}")

The click through rate confidence interval is: (4.03%, 5.17%)


The average click through rate for facebook ads is around 1.44%, so this ad out performed the average.

Lets quickly compare the industry standard of a 5% conversion rate confidence interval to our calculated confidence interval.

In [94]:
conversions = 50
link_clicks = 1000
confidence_interval = calculate_conversion_confidence_interval(conversions, link_clicks)
print(f"The standard conversion rate confidence interval is: {confidence_interval}")

The standard conversion rate confidence interval is: (3.65%, 6.35%)


In [100]:
lower_lim_conv = 239 * 0.0365
upper_lim_conv = 239 * 0.0635
print(f"The 95% confidence interval for amount of conversions is then between {round(lower_lim_conv,0)} and {round(upper_lim_conv,0)} people")

The 95% confidence interval for amount of conversions is then between 9.0 and 15.0 people


From this, we can see that the ad campaign was more succesful than the typical campaign, atleast by conversion rate.

# Conclusion and suggestions

Based on the findings from this analysis, several key insights have emerged regarding the ad's performance across demographics and viewing behaviors. First, it is clear that males viewed the ad significantly more than females, both in terms of reach and impressions, which was confirmed through statistical testing. Although there was some indication that the reach-to-impression ratio between genders may differ, the p-value of 0.08 suggests that further investigation is needed to draw firm conclusions. The age group 55-64 stood out as the most frequent viewers, with a significant 99% difference from the lowest group, indicating a strong preference for this demographic. In terms of viewing times, the ad was most frequently seen during early morning hours, lunch breaks, and late afternoon, aligning with typical routines of the older demographic. However, due to the relatively minor differences in impressions during the top viewing hours, more data is required to determine the most effective time for ad placements.

The ad also performed well in terms of conversion and click-through rates, both surpassing industry averages. The 30% conversion rate is particularly notable, but the relatively low impression-to-conversion rate (1.4%) suggests there may be opportunities for improvement in driving conversions directly from impressions.

**Conclusion**: The findings demonstrate that the ad effectively engages a male-dominated audience, particularly in the 55-64 age group, and achieves higher-than-average conversion rates. However, further research is necessary to explore the gender-based differences in engagement and to better understand the impact of viewing times on ad performance. Additionally, optimizing the impression-to-conversion pathway may increase the overall effectiveness of the campaign.

**Suggestions for Further Research**:
1. **Gender-Based Engagement**: Investigate the reasons behind the higher engagement rates among males, exploring factors such as ad content, targeting, and platform usage.
2. **Optimal Viewing Times**: Conduct an expanded analysis of viewing times to refine the understanding of peak engagement windows, especially across different age groups.
3. **Impression to Conversion Optimization**: Research strategies to improve the low impression-to-conversion rate, potentially focusing on ad creative, landing page optimization, or retargeting strategies.
4. **Demographic-Specific Content**: Examine the potential for tailoring ad content more specifically to the 55-64 age group to increase both impressions and conversions.
5. **Longitudinal Data**: Collect additional data over a longer period to track changes in behavior and engagement, allowing for a more accurate evaluation of trends.

These directions could offer valuable insights for optimizing future campaigns and ensuring more efficient use of advertising resources.